I have csv file with data, that have these columns: Year,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,OriginAirportName,OriginCity,OriginState,DestAirportID,DestAirportName,DestCity,DestState,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDelay,ArrDel15,Cancelled,
Cleanse the data by identifying null values and replacing them with an appropriate value (zero in this case).

In [2]:
# Import required libraries for data cleansing and analysis
import pandas as pd
import numpy as np
import os
import json

In [3]:
# First, let's see what files are available in the data directory
data_dir = '/workspaces/flight-delays/data'
if os.path.exists(data_dir):
    print(f"Files in {data_dir}:")
    for file in os.listdir(data_dir):
        print(f"  - {file}")
else:
    print(f"Data directory {data_dir} not found. Let's check the current directory:")
    for file in os.listdir('.'):
        if file.endswith('.csv'):
            print(f"  - {file}")

Files in /workspaces/flight-delays/data:
  - processed
  - flights.csv


In [4]:
# Load the flights.csv file directly
csv_path = '/workspaces/flight-delays/data/flights.csv'
df = pd.read_csv(csv_path)

print(f"Successfully loaded flights.csv with shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")

Successfully loaded flights.csv with shape: (271940, 20)

Columns: ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'Carrier', 'OriginAirportID', 'OriginAirportName', 'OriginCity', 'OriginState', 'DestAirportID', 'DestAirportName', 'DestCity', 'DestState', 'CRSDepTime', 'DepDelay', 'DepDel15', 'CRSArrTime', 'ArrDelay', 'ArrDel15', 'Cancelled']


In [5]:
# Data cleansing: identify null values and replace with 0
print("=== DATA CLEANSING ===")

# Check for null values before cleaning
print("Null values before cleaning:")
null_before = df.isnull().sum()
print(null_before[null_before > 0])

# Replace all null values with 0
df_clean = df.fillna(0)

# Verify cleaning was successful
print("\nNull values after cleaning:")
null_after = df_clean.isnull().sum()
print(null_after[null_after > 0])

if null_after.sum() == 0:
    print("\n✅ Success! All null values have been replaced with 0")
else:
    print(f"\n⚠️ Warning: {null_after.sum()} null values still remain")

print(f"\nCleaned dataset shape: {df_clean.shape}")

=== DATA CLEANSING ===
Null values before cleaning:
DepDel15    2761
dtype: int64

Null values after cleaning:
Series([], dtype: int64)

✅ Success! All null values have been replaced with 0

Cleaned dataset shape: (271940, 20)
Series([], dtype: int64)

✅ Success! All null values have been replaced with 0

Cleaned dataset shape: (271940, 20)


In [6]:
# Save the cleansed data to a new file
import os

# Create processed data directory if it doesn't exist
processed_dir = '/workspaces/flight-delays/data/processed'
os.makedirs(processed_dir, exist_ok=True)

# Save cleansed data
clean_csv_path = os.path.join(processed_dir, 'flights_clean.csv')
df_clean.to_csv(clean_csv_path, index=False)

print(f"✅ Cleansed data saved to: {clean_csv_path}")
print(f"Original data: {df.shape[0]:,} rows")
print(f"Clean data: {df_clean.shape[0]:,} rows")
print(f"Columns: {df_clean.shape[1]}")

# Show a sample of the cleaned data
print(f"\nSample of cleaned data:")
print(df_clean.head())

✅ Cleansed data saved to: /workspaces/flight-delays/data/processed/flights_clean.csv
Original data: 271,940 rows
Clean data: 271,940 rows
Columns: 20

Sample of cleaned data:
   Year  Month  DayofMonth  DayOfWeek Carrier  OriginAirportID  \
0  2013      9          16          1      DL            15304   
1  2013      9          23          1      WN            14122   
2  2013      9           7          6      AS            14747   
3  2013      7          22          1      OO            13930   
4  2013      5          16          4      DL            13931   

              OriginAirportName  OriginCity OriginState  DestAirportID  \
0           Tampa International       Tampa          FL          12478   
1      Pittsburgh International  Pittsburgh          PA          13232   
2  Seattle/Tacoma International     Seattle          WA          11278   
3  Chicago O'Hare International     Chicago          IL          11042   
4         Norfolk International     Norfolk          VA   

In [7]:
# Step 2: Feature Engineering for Delay Prediction Model
print("=== FEATURE ENGINEERING ===")

# Start with the clean data
df_model = df_clean.copy()

# Filter out cancelled flights (we're predicting delays, not cancellations)
print(f"Before filtering cancelled flights: {len(df_model):,} rows")
df_model = df_model[df_model['Cancelled'] == 0]
print(f"After filtering cancelled flights: {len(df_model):,} rows")

# Convert scheduled times to hour of day
df_model['DepHour'] = (df_model['CRSDepTime'] // 100).astype(int)
df_model['ArrHour'] = (df_model['CRSArrTime'] // 100).astype(int)

# Fix any hour values that might be 24+ (should be rare)
df_model['DepHour'] = df_model['DepHour'].clip(0, 23)
df_model['ArrHour'] = df_model['ArrHour'].clip(0, 23)

# Create time of day categories
def get_time_category(hour):
    if 6 <= hour <= 9:
        return 'morning_rush'
    elif 10 <= hour <= 16:
        return 'midday'
    elif 17 <= hour <= 20:
        return 'evening_rush'
    else:
        return 'night'

df_model['DepTimeCategory'] = df_model['DepHour'].apply(get_time_category)

# Create additional date features
df_model['IsWeekend'] = (df_model['DayOfWeek'].isin([6, 7])).astype(int)
df_model['Season'] = df_model['Month'].apply(lambda x: 
    'Winter' if x in [12, 1, 2] else
    'Spring' if x in [3, 4, 5] else
    'Summer' if x in [6, 7, 8] else
    'Fall'
)

print(f"\nFeature engineering completed!")
print(f"New columns added: DepHour, ArrHour, DepTimeCategory, IsWeekend, Season")
print(f"Target variable: ArrDel15 (1 = delayed >15 minutes)")

# Show distribution of target variable
delay_rate = df_model['ArrDel15'].mean()
print(f"\nOverall delay rate (>15 min): {delay_rate:.1%}")
print(f"Total flights for training: {len(df_model):,}")

=== FEATURE ENGINEERING ===
Before filtering cancelled flights: 271,940 rows
After filtering cancelled flights: 269,024 rows

Feature engineering completed!
New columns added: DepHour, ArrHour, DepTimeCategory, IsWeekend, Season
Target variable: ArrDel15 (1 = delayed >15 minutes)

Overall delay rate (>15 min): 20.7%
Total flights for training: 269,024

Feature engineering completed!
New columns added: DepHour, ArrHour, DepTimeCategory, IsWeekend, Season
Target variable: ArrDel15 (1 = delayed >15 minutes)

Overall delay rate (>15 min): 20.7%
Total flights for training: 269,024


In [8]:
# Prepare features for modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
import pickle

print("=== MODEL PREPARATION ===")

# Select features for the model
feature_columns = [
    'Year', 'Month', 'DayofMonth', 'DayOfWeek', 
    'OriginAirportID', 'DestAirportID', 
    'Carrier', 'DepHour', 'DepTimeCategory', 'IsWeekend', 'Season'
]

# Prepare the dataset
X = df_model[feature_columns].copy()
y = df_model['ArrDel15'].copy()

print(f"Features: {feature_columns}")
print(f"Target: ArrDel15 (arrival delay >15 minutes)")
print(f"Dataset size: {X.shape[0]:,} flights, {X.shape[1]} features")

# Handle categorical variables
categorical_features = ['Carrier', 'DepTimeCategory', 'Season']
label_encoders = {}

for col in categorical_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

print(f"Encoded categorical features: {categorical_features}")

# Time-based split: use earlier data for training, later for testing
# This mimics real-world prediction where we predict future flights
train_size = int(0.8 * len(X))
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

print(f"Training set: {len(X_train):,} flights")
print(f"Test set: {len(X_test):,} flights")
print(f"Training period delay rate: {y_train.mean():.1%}")
print(f"Test period delay rate: {y_test.mean():.1%}")

=== MODEL PREPARATION ===
Features: ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'OriginAirportID', 'DestAirportID', 'Carrier', 'DepHour', 'DepTimeCategory', 'IsWeekend', 'Season']
Target: ArrDel15 (arrival delay >15 minutes)
Dataset size: 269,024 flights, 11 features
Encoded categorical features: ['Carrier', 'DepTimeCategory', 'Season']
Training set: 215,219 flights
Test set: 53,805 flights
Training period delay rate: 20.8%
Test period delay rate: 20.5%


In [9]:
# Train delay prediction models
print("=== MODEL TRAINING ===")

# Model 1: Logistic Regression (simple, interpretable)
print("Training Logistic Regression model...")
lr_model = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
lr_model.fit(X_train, y_train)

# Model 2: Random Forest (more complex, handles interactions)
print("Training Random Forest model...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate both models
print("\n=== MODEL EVALUATION ===")

for name, model in [("Logistic Regression", lr_model), ("Random Forest", rf_model)]:
    print(f"\n{name} Results:")
    
    # Predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability of delay
    
    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    print(f"  Accuracy: {accuracy:.3f}")
    print(f"  ROC-AUC: {roc_auc:.3f}")
    print(f"  Avg predicted delay probability: {y_pred_proba.mean():.1%}")

# Choose the best model (Random Forest typically performs better)
best_model = rf_model
print(f"\n✅ Selected Random Forest as the final model")

# Show feature importance
feature_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': best_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nTop 5 Most Important Features:")
print(feature_importance.head())

=== MODEL TRAINING ===
Training Logistic Regression model...
Training Random Forest model...
Training Random Forest model...

=== MODEL EVALUATION ===

Logistic Regression Results:
  Accuracy: 0.599
  ROC-AUC: 0.641
  Avg predicted delay probability: 48.3%

Random Forest Results:

=== MODEL EVALUATION ===

Logistic Regression Results:
  Accuracy: 0.599
  ROC-AUC: 0.641
  Avg predicted delay probability: 48.3%

Random Forest Results:
  Accuracy: 0.795
  ROC-AUC: 0.695
  Avg predicted delay probability: 20.7%

✅ Selected Random Forest as the final model

Top 5 Most Important Features:
           Feature  Importance
5    DestAirportID    0.245576
4  OriginAirportID    0.236843
2       DayofMonth    0.142459
6          Carrier    0.127954
7          DepHour    0.111134
  Accuracy: 0.795
  ROC-AUC: 0.695
  Avg predicted delay probability: 20.7%

✅ Selected Random Forest as the final model

Top 5 Most Important Features:
           Feature  Importance
5    DestAirportID    0.245576
4  Origin

In [10]:
# Save the complete model pipeline for external use
print("=== SAVING MODEL FOR EXTERNAL USE ===")

# Create a complete pipeline object that includes preprocessing and model
model_pipeline = {
    'model': best_model,
    'label_encoders': label_encoders,
    'feature_columns': feature_columns,
    'categorical_features': categorical_features,
    'model_version': '1.0',
    'trained_date': pd.Timestamp.now().strftime('%Y-%m-%d'),
    'training_size': len(X_train),
    'test_accuracy': accuracy_score(y_test, best_model.predict(X_test)),
    'test_roc_auc': roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])
}

# Save to the server directory (matches your repo structure)
server_dir = '/workspaces/flight-delays/server'
os.makedirs(server_dir, exist_ok=True)

model_path = os.path.join(server_dir, 'model.pkl')
with open(model_path, 'wb') as f:
    pickle.dump(model_pipeline, f)

print(f"✅ Model saved to: {model_path}")
print(f"Model version: {model_pipeline['model_version']}")
print(f"Training date: {model_pipeline['trained_date']}")
print(f"Test accuracy: {model_pipeline['test_accuracy']:.3f}")
print(f"Test ROC-AUC: {model_pipeline['test_roc_auc']:.3f}")

# Test the saved model by loading it back
print(f"\n=== TESTING SAVED MODEL ===")
with open(model_path, 'rb') as f:
    loaded_pipeline = pickle.load(f)

print(f"Successfully loaded model from file!")
print(f"Required input features: {loaded_pipeline['feature_columns']}")

# Example prediction function for external applications
def predict_delay_probability(year, month, day_of_month, day_of_week, 
                            origin_airport_id, dest_airport_id, carrier, 
                            departure_hour):
    """
    Predict probability of arrival delay >15 minutes
    
    Returns: probability (0.0 to 1.0)
    """
    # Create input data
    input_data = pd.DataFrame({
        'Year': [year],
        'Month': [month], 
        'DayofMonth': [day_of_month],
        'DayOfWeek': [day_of_week],
        'OriginAirportID': [origin_airport_id],
        'DestAirportID': [dest_airport_id],
        'Carrier': [carrier],
        'DepHour': [departure_hour],
        'DepTimeCategory': [get_time_category(departure_hour)],
        'IsWeekend': [1 if day_of_week in [6, 7] else 0],
        'Season': ['Winter' if month in [12, 1, 2] else
                  'Spring' if month in [3, 4, 5] else
                  'Summer' if month in [6, 7, 8] else 'Fall']
    })
    
    # Apply same preprocessing
    for col in loaded_pipeline['categorical_features']:
        input_data[col] = loaded_pipeline['label_encoders'][col].transform(input_data[col].astype(str))
    
    # Predict
    probability = loaded_pipeline['model'].predict_proba(input_data[loaded_pipeline['feature_columns']])[0][1]
    return probability

# Test with example
example_prob = predict_delay_probability(
    year=2023, month=12, day_of_month=15, day_of_week=5,  # Friday in December
    origin_airport_id=12892, dest_airport_id=14771,  # Example airport pair
    carrier='AA', departure_hour=17  # American Airlines, 5 PM departure
)

print(f"\nExample prediction:")
print(f"Friday Dec 15, 2023, AA flight departing 5 PM")
print(f"Predicted delay probability: {example_prob:.1%}")

=== SAVING MODEL FOR EXTERNAL USE ===


✅ Model saved to: /workspaces/flight-delays/server/model.pkl
Model version: 1.0
Training date: 2025-08-28
Test accuracy: 0.795
Test ROC-AUC: 0.695

=== TESTING SAVED MODEL ===
Successfully loaded model from file!
Required input features: ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'OriginAirportID', 'DestAirportID', 'Carrier', 'DepHour', 'DepTimeCategory', 'IsWeekend', 'Season']
Successfully loaded model from file!
Required input features: ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'OriginAirportID', 'DestAirportID', 'Carrier', 'DepHour', 'DepTimeCategory', 'IsWeekend', 'Season']


ValueError: y contains previously unseen labels: 'Winter'

In [11]:
# Step 4: Create airports lookup file with names and IDs
print("=== CREATING AIRPORTS LOOKUP FILE ===")

# Load the cleaned data if not already available
try:
    df_clean
    print("Using df_clean from previous cells")
except NameError:
    print("Loading cleaned data from file...")
    clean_csv_path = '/workspaces/flight-delays/data/processed/flights_clean.csv'
    df_clean = pd.read_csv(clean_csv_path)
    print(f"Loaded cleaned data: {df_clean.shape}")

# Extract unique airports from both origin and destination columns
origin_airports = df_clean[['OriginAirportID', 'OriginAirportName']].rename(columns={
    'OriginAirportID': 'AirportID',
    'OriginAirportName': 'AirportName'
})

dest_airports = df_clean[['DestAirportID', 'DestAirportName']].rename(columns={
    'DestAirportID': 'AirportID', 
    'DestAirportName': 'AirportName'
})

# Combine and deduplicate
all_airports = pd.concat([origin_airports, dest_airports], ignore_index=True)

print(f"Total airport records before deduplication: {len(all_airports):,}")

# Remove duplicates and handle conflicts
# Group by AirportID and take the most frequent name for each ID
airports_grouped = all_airports.groupby('AirportID')['AirportName'].agg(['first', 'nunique']).reset_index()
airports_grouped.columns = ['AirportID', 'AirportName', 'NameVariations']

# Check for airports with multiple names
name_conflicts = airports_grouped[airports_grouped['NameVariations'] > 1]
if len(name_conflicts) > 0:
    print(f"Found {len(name_conflicts)} airports with multiple name variations:")
    print(name_conflicts.head())

# Create final clean airports list
airports_final = airports_grouped[['AirportID', 'AirportName']].copy()
airports_final = airports_final.sort_values('AirportName').reset_index(drop=True)

print(f"Final unique airports: {len(airports_final):,}")

# Save airports lookup file
airports_path = '/workspaces/flight-delays/data/airports.csv'
airports_final.to_csv(airports_path, index=False)

print(f"✅ Airports lookup saved to: {airports_path}")
print(f"\nSample of airports data:")
print(airports_final.head(10))

# Also create a JSON version for web applications
airports_json_path = '/workspaces/flight-delays/data/airports.json'
airports_dict = airports_final.to_dict('records')
with open(airports_json_path, 'w') as f:
    json.dump(airports_dict, f, indent=2)

print(f"✅ Airports lookup also saved as JSON: {airports_json_path}")

# Summary statistics
print(f"\n=== SUMMARY ===")
print(f"✅ Data cleansing completed: {len(df_clean):,} flights")
print(f"✅ Model trained and saved: {model_path}")
print(f"✅ Airports lookup created: {len(airports_final):,} unique airports")
print(f"✅ All files ready for external applications!")

print(f"\nFiles created:")
print(f"  - {clean_csv_path} (cleaned flight data)")
print(f"  - {model_path} (trained model)")
print(f"  - {airports_path} (airports CSV)")
print(f"  - {airports_json_path} (airports JSON)")

=== CREATING AIRPORTS LOOKUP FILE ===
Using df_clean from previous cells
Total airport records before deduplication: 543,880


Final unique airports: 70
✅ Airports lookup saved to: /workspaces/flight-delays/data/airports.csv

Sample of airports data:
   AirportID                                        AirportName
0      10140                  Albuquerque International Sunport
1      10423                   Austin - Bergstrom International
2      10821  Baltimore/Washington International Thurgood Ma...
3      10800                                           Bob Hope
4      10529                              Bradley International
5      10792                      Buffalo Niagara International
6      11057                    Charlotte Douglas International
7      13232                       Chicago Midway International
8      13930                       Chicago O'Hare International
9      11193         Cincinnati/Northern Kentucky International
✅ Airports lookup also saved as JSON: /workspaces/flight-delays/data/airports.json

=== SUMMARY ===
✅ Data cleansing completed: 271,940 flights
✅ Model trained and saved: /